In [26]:
fn = get_filename()
write_slopes(fn, get_slopes(fn))

Saved to D:/Documents/MEGAsync/Uni/Semester 7/Zurbriggen lab/Bachelor Thesis/SEAP tool/test - Copy - Copy - Copy.xlsx


In [25]:
#imports and methods
#you might have to install rdp with 'pip install rdp'
import pandas as pd
import tkinter as tk
from tkinter import filedialog as fd
from matplotlib import pyplot as plt
import numpy as np
from rdp import rdp
from openpyxl import load_workbook


def get_filename():
    root = tk.Tk()
    root.withdraw()
    filename = fd.askopenfilename()
    return filename
    root.destroy()

def get_slopes(filename, cb = True, res = 'funct', eps = 0.15, write = False):
    #parameters:
    #filename: name of the file to analyze, use get_filename()
    #cb: when True, also copy the result to the clipboard
    #res: chose the form of the results: 'funct' for funct formulars, 'slopes' for numeric values and 's_e' for
    #-- a list of start- and endpoints
    #create list to append funct formulars
    funct = list()
    #and one to add the slopes as numeric values
    slopes = list()
    #and one to add the start and end points
    s_e = list()
    #create the x values for the full measurement - a list with 60 entries, going from 0 to 59 in increases of 1
    x = np.linspace(0,59,60)
    #import the data series from the funct template
    pdf = pd.read_excel(filename, sheet_name = 'SEAP vertical transformation', usecols = 'C:CT', skiprows = 35)
    #create new instance of tkinter, used for accessing the clipboard
    r = tk.Tk()
    r.withdraw()
    #clear the clipboard
    r.clipboard_clear()
    #iterate over the columns in the dataframe, representing a series of A405 measurements
    for name, values in pdf.iteritems():
        #converting the index of a well to an funct column index (+3 as the first 3 columns in the template are unused)
        col = col_int_to_char(pdf.columns.get_loc(name)+3)
        #transform the values to pairs (two dimensional coordinates)
        vst = np.vstack((x, np.array(values))).T
        #apply the RDP algorithm on the set of data
        vst = rdp(vst, epsilon = eps)
        #retreive the starting and ending point of the first RDP segment
        s = int(vst[0][0])
        e = int(vst[1][0])
        #move the start to remove all 'dropping values' from the start to optimize the outcome
        while values[s]>values[s+1]: 
            s = s+1
        #calculate the slope from the linear segment
        slope = (values[e]-values[s])/(e-s)
        #generate the funct formular which calculates the slope in the template
        code = "=SLOPE({c}{s}:{c}{e},$A${s}:$A${e})".format(c = col, s = s+37, e = e+37)
        #when cb is True, append it to the clipboard and append a tabulator for simplifying the pasting
        if cb: r.clipboard_append(code+'\t')
        #append the results to their respective lists
        funct.append(code)
        slopes.append(slope)
        s_e.append((s,e))
    #update the content of the clipboard
    r.update()
    #if write == True, the slopes will be directly written to the file
    if write: write_slopes(filename, funct)
    #return the slopes in the selected way
    if res == 'funct': return funct
    if res == 'slopes': return slopes
    if res == 's_e': return s_e


def plot_slopes(filename, plt_nr = 24, s_e = False):
    #parameters:
    #filename: name of the file to plot from, use get_filename()    
    #n: number of plots to be generated, it must be a divider of 96
    #s_e: if the output of get_slopes(...res='s_e') should be used, pass it here
    color = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99']*9
    if 96%plt_nr!=0: 
        print('96%plt_nr must be 0!')
        return None
    pdf = pd.read_excel(filename, sheet_name = 'SEAP vertical transformation', usecols = 'C:CT', skiprows = 35)
    if not s_e: s_e = get_slopes(filename, res='s_e')
    for i in range(0,plt_nr):
        if plt_nr<12:  #yeah yeah, just don't comment it! :E
            f = plt.figure()
            f.set_figwidth(16)
            f.set_figheight(8)
        for j in range(0,int(96/plt_nr)):
            col = pdf.iloc[:,int(i*(96/plt_nr)+j)]
            x = np.linspace(10*j,59+10*j,60)
            y = np.array(col)
            plt.plot(x, y, c = color[j])
            curse = s_e[int(i*(96/plt_nr)+j)]
            x = np.linspace(curse[0]+10*j, curse[1]+10*j, curse[1]-curse[0])
            y = np.linspace(col[curse[0]], col[curse[1]], curse[1]-curse[0])
            plt.plot(x,y, c = color[j], lw = 4, alpha = .5)
        plt.title(pdf.columns[int(i*(96/plt_nr))]+'-'+pdf.columns[int((i*(96/plt_nr)+j))])
        plt.show()

def write_slopes(filename, slopes):
    slopes = pd.DataFrame([slopes])
    book = load_workbook(filename)
    writer = pd.ExcelWriter(filename, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    slopes.to_excel(writer, "SEAP vertical transformation", header = False, index = False, startrow = 30, startcol = 2)
    writer.save()
    print('Saved to '+filename)

def col_int_to_char(col_nr): #makes funct columns out of numbers (i.e. 96 -> 'CT')
    ch = ''
    while col_nr > 0:
        col_nr,r = divmod(col_nr-1, 26)
        ch = chr(r+ord('A'))+ch
    return ch